In [1]:
from pyspark.sql import SparkSession, functions as f, types as t

In [2]:
spark = SparkSession.builder.appName('lect_13_home_task').getOrCreate()

22/10/22 23:58:45 WARN Utils: Your hostname, anatoliidomanskii-System-Product-Name resolves to a loopback address: 127.0.1.1; using 192.168.1.25 instead (on interface enp5s0)
22/10/22 23:58:45 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/22 23:58:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
actor_df = spark.read.csv('./data/actor.csv', header=True, inferSchema=True)
address_df = spark.read.csv('./data/address.csv', header=True, inferSchema=True)
category_df = spark.read.csv('./data/category.csv', header=True, inferSchema=True)
city_df = spark.read.csv('./data/city.csv', header=True, inferSchema=True)
country_df = spark.read.csv('./data/country.csv', header=True, inferSchema=True)
customer_df = spark.read.csv('./data/customer.csv', header=True, inferSchema=True)
film_df = spark.read.csv('./data/film.csv', header=True, inferSchema=True)
film_actor_df = spark.read.csv('./data/film_actor.csv', header=True, inferSchema=True)
film_category_df = spark.read.csv('./data/film_category.csv', header=True, inferSchema=True)
inventory_df = spark.read.csv('./data/inventory.csv', header=True, inferSchema=True)
language_df = spark.read.csv('./data/language.csv', header=True, inferSchema=True)
payment_df = spark.read.csv('./data/payment.csv', header=True, inferSchema=True)
rental_df = spark.read.csv('./data/rental.csv', header=True, inferSchema=True)
staff_df = spark.read.csv('./data/staff.csv', header=True, inferSchema=True)
store_df = spark.read.csv('./data/store.csv', header=True, inferSchema=True)

# Домашнє завдання на тему Spark SQL

Задачі з домашнього завдання на SQL потрібно розвʼязати за допомогою Spark SQL DataFrame API.

- Дампи таблиць знаходяться в папці `data`. Датафрейми таблиць вже створені в клітинці вище.
- Можете створювати стільки нових клітинок, скільки вам необхідно.
- Розвʼязок кожної задачі має бути відображений в самому файлі (використати метод `.show()`)

**Увага!**
Використовувати мову запитів SQL безпосередньо забороняється, потрібно використовувати виключно DataFrame API!


1.
Вивести кількість фільмів в кожній категорії.
Результат відсортувати за спаданням.

In [4]:
film_category_df.\
    join(category_df, film_category_df.category_id==category_df.category_id, 'left').\
    groupby(category_df.name).\
    agg(f.count('film_id').alias('num_of_films')).\
    orderBy(f.desc('num_of_films')).\
    show()

+-----------+------------+
|       name|num_of_films|
+-----------+------------+
|     Sports|          74|
|    Foreign|          73|
|     Family|          69|
|Documentary|          68|
|  Animation|          66|
|     Action|          64|
|        New|          63|
|      Drama|          62|
|      Games|          61|
|     Sci-Fi|          61|
|   Children|          60|
|     Comedy|          58|
|     Travel|          57|
|   Classics|          57|
|     Horror|          56|
|      Music|          51|
+-----------+------------+



2.
Вивести 10 акторів, чиї фільми брали на прокат найбільше.
Результат відсортувати за спаданням.

In [5]:
rental_df.\
    join(inventory_df, rental_df.inventory_id==inventory_df.inventory_id, 'left').\
    join(film_actor_df, inventory_df.film_id==film_actor_df.film_id, 'left').\
    join(actor_df, film_actor_df.actor_id==actor_df.actor_id).\
    where(actor_df.actor_id.isNotNull()).\
    groupby(actor_df.first_name, actor_df.last_name).\
    agg(f.count(rental_df.rental_id).alias('qty')).\
    orderBy(f.desc('qty')).\
    limit(10).\
    show()

+----------+-----------+---+
|first_name|  last_name|qty|
+----------+-----------+---+
|     SUSAN|      DAVIS|825|
|      GINA|  DEGENERES|753|
|   MATTHEW|     CARREY|678|
|      MARY|     KEITEL|674|
|    ANGELA|WITHERSPOON|654|
|    WALTER|       TORN|640|
|     HENRY|      BERRY|612|
|     JAYNE|      NOLTE|611|
|       VAL|     BOLGER|605|
|    SANDRA|     KILMER|604|
+----------+-----------+---+



3.
Вивести категорія фільмів, на яку було витрачено найбільше грошей
в прокаті

In [6]:
payment_df.\
    join(rental_df, payment_df.rental_id==rental_df.rental_id, 'left').\
    join(inventory_df, rental_df.inventory_id==inventory_df.inventory_id, 'left').\
    join(film_category_df, inventory_df.film_id==film_category_df.film_id, 'left').\
    join(category_df, film_category_df.category_id==category_df.category_id, 'left').\
    groupby(category_df.name).\
    agg(f.sum(payment_df.amount).alias('payments')).\
    orderBy(f.desc('payments')).\
    limit(1).\
    show()

+------+-----------------+
|  name|         payments|
+------+-----------------+
|Sports|5314.209999999847|
+------+-----------------+



4.
Вивести назви фільмів, яких не має в inventory.
Запит має бути без оператора IN

In [7]:
film_df.\
    join(inventory_df, inventory_df.film_id==film_df.film_id, 'left')\
    .where(inventory_df.inventory_id.isNull())\
    .select(film_df.title)\
    .show()

+--------------------+
|               title|
+--------------------+
|      ALICE FANTASIA|
|         APOLLO TEEN|
|      ARGONAUTS TOWN|
|       ARK RIDGEMONT|
|ARSENIC INDEPENDENCE|
|   BOONDOCK BALLROOM|
|       BUTCH PANTHER|
|       CATCH AMISTAD|
| CHINATOWN GLADIATOR|
|      CHOCOLATE DUCK|
|COMMANDMENTS EXPRESS|
|    CROSSING DIVORCE|
|     CROWDS TELEMARK|
|    CRYSTAL BREAKING|
|          DAZED PUNK|
|DELIVERANCE MULHO...|
|   FIREHOUSE VIETNAM|
|       FLOATS GARDEN|
|FRANKENSTEIN STRA...|
|  GLADIATOR WESTWARD|
+--------------------+
only showing top 20 rows



5.
Вивести топ 3 актори, які найбільше зʼявлялись в категорії фільмів “Children”

In [82]:
film_df.\
    join(film_actor_df, film_df.film_id==film_actor_df.film_id, 'left').\
    join(film_category_df, film_df.film_id==film_category_df.film_id, 'left').\
    join(actor_df, actor_df.actor_id==film_actor_df.actor_id, 'left').\
    where(film_category_df.category_id == 3).\
    groupby(actor_df.first_name, actor_df.last_name).\
    agg(f.count(film_df.film_id).alias('count_of_films')).\
    orderBy(f.desc('count_of_films')).\
    limit(3).\
    show()

+----------+---------+--------------+
|first_name|last_name|count_of_films|
+----------+---------+--------------+
|     HELEN|   VOIGHT|             7|
|     SUSAN|    DAVIS|             6|
|     RALPH|     CRUZ|             5|
+----------+---------+--------------+



6.
Вивести міста з кількістю активних та неактивних клієнтів
(в активних customer.active = 1).
Результат відсортувати за кількістю неактивних клієнтів за спаданням.

In [53]:
customer_df.\
    join(address_df, address_df.address_id==customer_df.address_id, 'left').\
    withColumn('No_active', f.when(customer_df.active==0, 1).otherwise(0)).\
    groupby('city_id').agg(f.sum('active').alias('Active_cust'), f.sum('No_active').alias('No_active_cust')).\
    orderBy(f.desc('No_active_cust')).\
    show()

+-------+-----------+--------------+
|city_id|Active_cust|No_active_cust|
+-------+-----------+--------------+
|    111|          0|             1|
|    139|          0|             1|
|    283|          0|             1|
|    578|          0|             1|
|    356|          0|             1|
|     24|          0|             1|
|    125|          0|             1|
|    281|          0|             1|
|    407|          0|             1|
|     57|          0|             1|
|    259|          0|             1|
|    554|          0|             1|
|    577|          0|             1|
|    495|          0|             1|
|    512|          0|             1|
|    463|          1|             0|
|    148|          1|             0|
|    471|          1|             0|
|    243|          1|             0|
|    496|          1|             0|
+-------+-----------+--------------+
only showing top 20 rows

